# 多项式回归 (Polynomial Regression)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解多项式回归的原理和应用场景
- 掌握如何使用多项式特征扩展线性模型
- 学会使用Scikit-learn的PolynomialFeatures
- 理解模型复杂度与过拟合的关系

## 📋 前置知识

- 线性回归基础
- NumPy基本操作
- Scikit-learn基础

## ⏱️ 预计时间

20-30分钟

---

## 📖 第1部分：理论背景

### 什么是多项式回归？

多项式回归是一种通过添加**多项式特征**来扩展线性模型的技术。虽然数据之间的关系可能是非线性的，但我们可以通过特征变换将其转化为线性问题。

### 数学模型

对于简单的一元二次多项式回归：

$$y = \theta_0 + \theta_1 x + \theta_2 x^2$$

我们可以将其视为在扩展特征空间 $[x, x^2]$ 上的线性回归：

$$y = \theta_0 + \theta_1 x_1 + \theta_2 x_2$$

其中 $x_1 = x$, $x_2 = x^2$

### 更一般的形式

对于 $d$ 次多项式：

$$y = \theta_0 + \theta_1 x + \theta_2 x^2 + ... + \theta_d x^d$$

### 核心思想

> 💡 **关键洞察**: 多项式回归的本质是**特征工程** —— 通过创建原始特征的多项式组合，将非线性问题转化为线性问题。

### 适用场景

- 数据呈现明显的曲线关系
- 简单线性回归拟合效果不佳
- 需要捕获特征之间的交互作用

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

# 数值计算
import numpy as np

# 可视化
import matplotlib.pyplot as plt

# 机器学习
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 数据生成参数
N_SAMPLES = 100
NOISE_LEVEL = 1.0

# 真实参数 (y = 0.5x² + x + 2)
TRUE_COEF_0 = 2.0    # 常数项
TRUE_COEF_1 = 1.0    # 一次项系数
TRUE_COEF_2 = 0.5    # 二次项系数

# 可视化配置
FIGURE_SIZE = (12, 5)
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 库导入和配置完成")

### 步骤2: 生成非线性数据

我们生成符合二次关系的数据：$y = 0.5x^2 + x + 2 + \epsilon$

In [ ]:
# ============================================================
# 生成非线性数据
# ============================================================

# 生成特征X：范围[-3, 3]
X = 6 * np.random.rand(N_SAMPLES, 1) - 3

# 生成目标y：y = 0.5x² + x + 2 + 噪声
y = TRUE_COEF_2 * X**2 + TRUE_COEF_1 * X + TRUE_COEF_0 + NOISE_LEVEL * np.random.randn(N_SAMPLES, 1)

# 数据检查
print("✓ 数据生成完成")
print(f"  样本数量: {N_SAMPLES}")
print(f"  X形状: {X.shape}")
print(f"  y形状: {y.shape}")
print(f"  X范围: [{X.min():.2f}, {X.max():.2f}]")
print(f"  y范围: [{y.min():.2f}, {y.max():.2f}]")
print(f"\n真实参数:")
print(f"  常数项 (θ0): {TRUE_COEF_0}")
print(f"  一次项 (θ1): {TRUE_COEF_1}")
print(f"  二次项 (θ2): {TRUE_COEF_2}")

### 步骤3: 数据可视化

观察数据分布，确认其非线性特性：

In [ ]:
# ============================================================
# 数据可视化
# ============================================================

plt.figure(figsize=(10, 6))

# 绘制散点图
plt.scatter(X, y, alpha=0.6, s=50, 
            edgecolors='k', linewidth=0.5, 
            label='数据点', color='steelblue')

# 绘制真实曲线
X_line = np.linspace(-3, 3, 100).reshape(-1, 1)
y_true = TRUE_COEF_2 * X_line**2 + TRUE_COEF_1 * X_line + TRUE_COEF_0
plt.plot(X_line, y_true, 'g--', linewidth=2, label='真实曲线')

plt.xlabel('X', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('非线性数据分布 (二次关系)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.legend()
plt.tight_layout()
plt.show()

print("✓ 可以看到数据呈现明显的抛物线形状")

---

### 步骤4: 使用PolynomialFeatures创建多项式特征

`PolynomialFeatures`会将原始特征扩展为多项式特征：
- 输入: $[x]$
- 输出 (degree=2): $[x, x^2]$ 或 $[1, x, x^2]$ (include_bias=True)

In [ ]:
# ============================================================
# 创建多项式特征
# ============================================================

# 创建2次多项式特征转换器
# include_bias=False: 不包含常数项1（LinearRegression会自动添加）
poly_features = PolynomialFeatures(degree=2, include_bias=False)

# 转换特征
X_poly = poly_features.fit_transform(X)

print("多项式特征转换:")
print(f"  原始特征形状: {X.shape}")
print(f"  转换后形状: {X_poly.shape}")
print(f"\n转换示例 (前5个样本):")
print(f"{'原始X':>10} {'X':>10} {'X²':>10}")
print("-" * 32)
for i in range(5):
    print(f"{X[i][0]:>10.4f} {X_poly[i][0]:>10.4f} {X_poly[i][1]:>10.4f}")

print(f"\n特征名称: {poly_features.get_feature_names_out()}")

### 步骤5: 使用线性回归拟合多项式特征

In [ ]:
# ============================================================
# 线性回归拟合
# ============================================================

# 创建并训练线性回归模型
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

# 获取参数
print("="*50)
print("多项式回归结果")
print("="*50)
print(f"{'参数':<15} {'估计值':>12} {'真实值':>12} {'误差':>12}")
print("-"*50)
print(f"{'常数项 (θ0)':<15} {lin_reg.intercept_[0]:>12.4f} {TRUE_COEF_0:>12.4f} {abs(lin_reg.intercept_[0] - TRUE_COEF_0):>12.4f}")
print(f"{'一次项 (θ1)':<15} {lin_reg.coef_[0][0]:>12.4f} {TRUE_COEF_1:>12.4f} {abs(lin_reg.coef_[0][0] - TRUE_COEF_1):>12.4f}")
print(f"{'二次项 (θ2)':<15} {lin_reg.coef_[0][1]:>12.4f} {TRUE_COEF_2:>12.4f} {abs(lin_reg.coef_[0][1] - TRUE_COEF_2):>12.4f}")

print(f"\n拟合的方程: y = {lin_reg.coef_[0][1]:.4f}x² + {lin_reg.coef_[0][0]:.4f}x + {lin_reg.intercept_[0]:.4f}")

### 步骤6: 模型评估和可视化

In [ ]:
# ============================================================
# 模型评估
# ============================================================

# 预测
y_pred = lin_reg.predict(X_poly)

# 计算评估指标
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print("模型评估:")
print(f"  MSE: {mse:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R² Score: {r2:.4f}")

In [ ]:
# ============================================================
# 拟合结果可视化
# ============================================================

# 生成平滑曲线用于绘图
X_line = np.linspace(-3, 3, 100).reshape(-1, 1)
X_line_poly = poly_features.transform(X_line)
y_line_pred = lin_reg.predict(X_line_poly)

plt.figure(figsize=FIGURE_SIZE)

# 左图：拟合结果
plt.subplot(1, 2, 1)
plt.scatter(X, y, alpha=0.6, s=50, edgecolors='k', linewidth=0.5, label='数据点')
plt.plot(X_line, y_line_pred, 'r-', linewidth=2, label='拟合曲线')
plt.xlabel('X', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('多项式回归拟合结果', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# 右图：预测值 vs 真实值
plt.subplot(1, 2, 2)
plt.scatter(y, y_pred, alpha=0.6, s=50, edgecolors='k', linewidth=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2, label='理想拟合线')
plt.xlabel('真实值', fontsize=12)
plt.ylabel('预测值', fontsize=12)
plt.title(f'预测值 vs 真实值 (R² = {r2:.4f})', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 🔬 第3部分：不同多项式次数对比

让我们比较不同次数的多项式拟合效果，观察欠拟合和过拟合现象：

In [ ]:
# ============================================================
# 不同多项式次数对比
# ============================================================

degrees = [1, 2, 10, 20]  # 不同的多项式次数

plt.figure(figsize=(15, 4))

for i, degree in enumerate(degrees):
    plt.subplot(1, 4, i+1)
    
    # 创建多项式特征
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly_d = poly.fit_transform(X)
    
    # 训练模型
    model = LinearRegression()
    model.fit(X_poly_d, y)
    
    # 预测
    X_line_poly_d = poly.transform(X_line)
    y_line_d = model.predict(X_line_poly_d)
    
    # 计算R²
    y_pred_d = model.predict(X_poly_d)
    r2_d = r2_score(y, y_pred_d)
    
    # 绘图
    plt.scatter(X, y, alpha=0.5, s=30, edgecolors='k', linewidth=0.3)
    plt.plot(X_line, y_line_d, 'r-', linewidth=2)
    plt.xlabel('X')
    plt.ylabel('y')
    plt.title(f'Degree = {degree}\nR² = {r2_d:.4f}', fontsize=11)
    plt.ylim(y.min() - 2, y.max() + 2)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("观察结论:")
print("  - Degree=1 (线性): 欠拟合，无法捕获数据的曲线关系")
print("  - Degree=2 (二次): 良好拟合，与真实模型一致")
print("  - Degree=10/20: 开始过拟合，曲线波动大")

---

## 📝 总结

### 关键要点

1. ✅ **多项式回归**通过添加多项式特征将非线性问题转化为线性问题
2. ✅ 使用`PolynomialFeatures`可以方便地生成多项式特征
3. ✅ 多项式次数的选择很重要：
   - 次数过低：**欠拟合** (高偏差)
   - 次数过高：**过拟合** (高方差)
4. ✅ 实际应用中需要通过交叉验证选择合适的多项式次数

### 注意事项

> ⚠️ **警告**: 高次多项式容易过拟合，特别是在数据量较少时

> 💡 **建议**: 使用正则化（Ridge、Lasso）可以有效缓解过拟合问题

### 下一步学习

- 📘 学习曲线分析
- 📙 岭回归 (Ridge Regression)
- 📕 Lasso回归

### 练习题

1. 尝试使用3次多项式拟合，观察效果
2. 增加噪声水平，观察模型表现变化
3. 使用交叉验证选择最佳多项式次数

## 📚 参考资料

- [Scikit-learn: PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)
- 《机器学习实战》第4章